In [ ]:
from __future__ import print_function
from glob import glob
from ROOT import vector, RDataFrame, RDF, TFile, TH1F, TH2F, gInterpreter, TMath
import ROOT
import sys
import yaml
from bff_processor.cpp_function import def_cpp
from bff_processor.utils import toVector, get_nEvents
from bff_processor.df_definitions import *
import pandas as pd
from time import perf_counter
import uproot

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.assets.bff_skimmer import *

In [ ]:
# Compile functions in c++ for the dataframe
def_cpp()

In [ ]:
#set up multithreading
multiThreading = False
if multiThreading: ROOT.ROOT.EnableImplicitMT()
RDFrame = RDataFrame

In [ ]:
#set up metadata for files
era = '2016'
deepflavour = False
deepcsv = True
extras = False
csv_v_flavor=False
if era == "2016":
    fname = "samplesCR_2016_Apr2020.yml"
    bDiscValue = 0.6321
    if deepflavour:
        fname = 'samplesCR_2016_Apr2020-deepflavour.yml'
    if extras:
        fname = 'samplesCR_2016_Apr2020_extras.yml'
    if csv_v_flavor: 
        fname= 'samplesCR_2016_Apr2020-csv_v_flavor.yml'
if era == "2017":
    fname = "samplesCR_2017_Apr2020.yml"
    bDiscValue = 0.4941
    if deepflavour:
        bDiscValue = .3033
        fname = "samplesCR_2017_Apr2020_deepflavour.yml"
    if deepcsv:
        bDiscValue = .4941
        fname = "samplesCR_2017_Apr2020_deepcsv.yml"
if era == "2018":
    fname = "samplesCR_2018_Apr2020-hem.yml"
    bDiscValue = 0.4184


sampleDir = "samples"
outname = fname.replace('.yml','.root')
fname = "{}/{}".format(sampleDir, fname)
outname, fname

In [ ]:
class sample_processor():
    '''Class that takes file name from yml and helps manage samples.'''
    def __init__(self,file_name,outname,bDiscValue,is_inclusive=0):
        #load config
        self.file_name = file_name
        with open(file_name,'r') as f:
            self.sample_dict = yaml.load(f, Loader=yaml.FullLoader)
        #setup outfile
        self.outname = outname
        self.out = TFile(outname, 'recreate')
        self.outdirs_dict = {}
        for sample in self.samples():
            name = sample['name']
            self.outdirs_dict[name] = self.out.mkdir(name)
        self.lumi = self.sample_dict['lumi']
        if 'path' in self.sample_dict:
            self.path = self.sample_dict['path']
        else: 
            self.path = ''
        #get and write lumi info
        hlumi = TH1F("lumi", "lumi", 1, 0, 1)
        hlumi.SetDirectory(self.out)
        hlumi.SetBinContent(1, self.lumi)
        hlumi.Write()
        self.bDiscValue = bDiscValue
        self.is_inclusive = is_inclusive
    def samples(self):
        return self.sample_dict['samples']
    def sample_names(self):
        return [s['name'] for s in self.samples()]
    def close(self):
        self.out.Close()
    def __repr__(self):
        text_dict = {"fn":self.file_name,
                     "on":self.outname, 
                     "lumi":self.lumi,
                    "samples": self.sample_names()}
        return '''from {fn} to {on}\nlumi: {lumi}\nSamples {samples}'''.format(**text_dict)

In [ ]:
# create instance of sample manager class 
sp = sample_processor(fname, outname, bDiscValue)
print(sp)

In [ ]:
# this creates long list of or statements for all permuations to select for events that are present in at least one region 
rs = ["CR10", "CR11", "CR12", "CR13", "CR14", "CR20", "CR21", "CR22", "CR23", "CR24", "SR1", "SR2"]
JERC_var = ['nom','jerUp','jerDown','jesTotalUp','jesTotalDown']
string = ""
for jv in JERC_var:
    for r in rs:
        string += "{}_{} or ".format(r,jv)
mc_region = string[:-3]
JERC_var = ['nom']
string = ""
for jv in JERC_var:
    for r in rs:
        string += "{}_{} or ".format(r,jv)
data_region = string[:-3]
mc_region

In [ ]:
def process_sample(sp,sample,era,verbose=1):
    '''Process each sample and produced csv.'''
    #get metadata
    name,xsec,nevts = sample['name'],sample['xsec'],sample['nevts']
    ismc,fileglob,bTagEff = int(sample['ismc']),sample['fileglob'],sample['bTagEff']
    fileglob = fileglob.format(path=sp.path)
    bTagEff = bTagEff.format(path=sp.path)
    #make file glob
    #files = toVector('string', glob(fileglob))
    files = glob(fileglob)
    print(fileglob)
    #remove efficency files and DeepFlavour
    #files = list(filter(lambda x: "DeepFlavour" not in x, files))
    #files = list(filter(lambda x: "_Eff" not in x, files))
    # get n events from skim if 0
    if not nevts:
        nevts = get_nEvents(files)
        print(nevts)
    sample_weight = float(xsec)*sp.lumi/float(nevts)
    print(float(xsec), sp.lumi, float(nevts), sample_weight)
    if verbose: print("name: {} , xsec: {}, nevents: {} ismc: {}".format(name,xsec,nevts,ismc))
    #set up btagging and puid sf files
    bTagFile, PUIDSFfile = setup_btag_puid(ismc, era, bTagEff)
    #make rdf
    df = RDFrame('Events', files)
    #print([x for x in df.GetColumnNames()])
    df = df.Filter("DiLepMass>100", "mass_cut")
    if ismc:
        df = df.Filter(mc_region, "in_region")
    else:
        df = df.Filter(data_region, "in_region")
    df = def_good_leptons(df, ismc)
    df = def_good_jet(df,ismc, bDiscValue)
    df = def_HLT(df, ismc, era)
    df = def_sf_and_weight(df,ismc, sp.is_inclusive, name, sample_weight)
    df = def_lep_selections(df)
    # EE L1 prefiring test https://twiki.cern.ch/twiki/bin/viewauth/CMS/ExoPreapprovalChecklist
    df = df.Define("EE_L1_range", "GoodJetPt > 100 && GoodJetEta > 2.25 && GoodJetEta < 3.0")
    df = df.Define("EE_L1_prefire_test", "max_vec(EE_L1_range)!=0")

    # HEEM Veto (veto events with jet in it)
    #df = df.Define("Jet_in_HEM", "(-3 < Jet_eta && -1.3 > Jet_eta) && (-1.57 < Jet_phi && -.87 > Jet_phi")
    #df = df.Define("HEM_event_veto", "max_vec(Jet_in_HEEM)!=0")
    lcolumns = columns
    if not ismc:
        lcolumns = columns_data
    #save as CSV
    df_np = df.AsNumpy(lcolumns)
    df_df = pd.DataFrame(df_np)
    df_df.to_csv('data/tw_{}_{}.csv'.format(era,name))
    hem_columns = ['GoodJetEta','GoodJetPhi','GoodJetPt','GoodBJet','GoodMuonPt','GoodMuonEta','GoodMuonPhi','GoodElePt','GoodEleEta','GoodElePhi','DiLepMass','HTLT_nom','RelMET_nom','TMB_nom','SR2_nom','SR1_nom','CR10_nom','CR11_nom','CR12_nom','CR13_nom','CR14_nom','SR2_nom','CR20_nom','CR21_nom','CR22_nom','CR23_nom','CR24_nom',
                            'MET_phi','MET_pt',]
    if not ismc: hem_columns.append('run')
    df.Snapshot("Events", "data/tw_{}_{}.root".format(era,name), hem_columns)
    return name,df, fileglob

In [ ]:
for sample in sp.samples():
    name = sample['name']
    #if not 'ZToEE_M_800_1400_flavour' in name: continue
    #if not (('250' in name) or ('300' in name) or ('450' in name)): continue
    #if 'data' in name: continue
    print(name)
    start_time = perf_counter()
    name,df,fileglob = process_sample(sp,sample,era, verbose=1)
    #count = df.Count()
    end_time = perf_counter()
    try:
        print(name,end_time-start_time)
    except:
        print(name,end_time-start_time)
sp.close()